In [2]:
#import packages
import pandas as pd
import numpy as np
from collections import Counter
import IPython.core.display as ipd
import os
import sys
import time
import matplotlib.pyplot as plt

#set configurations
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [14]:
genuines= [1,1,0,0,1]
impostors= [1,0,0,0,0]

In [16]:
[1 if g==1 else 0 for i, g in enumerate(genuines)]

[1, 1, 0, 0, 1]

In [3]:
#dataset
DSL = pd.read_csv('dados/DSL-StrongPasswordData.csv')
DSL.head(100)
DSL = DSL.drop(['sessionIndex', 'rep'], axis=1)

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,0.2212,0.1043,0.1417,1.1885,1.0468,0.1146,1.6055,1.4909,0.1067,0.7590,0.6523,0.1016,0.2136,0.1120,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,0.1357,0.0449,0.0829,1.1970,1.1141,0.0689,0.7822,0.7133,0.1570,0.7877,0.6307,0.1066,0.1684,0.0618,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,0.1542,0.0721,0.0808,1.0408,0.9600,0.0892,0.6203,0.5311,0.1454,0.7195,0.5741,0.1365,0.2931,0.1566,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,0.2038,0.0998,0.0900,1.0556,0.9656,0.0913,1.2564,1.1651,0.1454,0.7550,0.6096,0.0956,0.1530,0.0574,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,0.1589,0.0686,0.0805,0.8629,0.7824,0.0742,0.8955,0.8213,0.1243,0.7632,0.6389,0.0430,0.1975,0.1545,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818
5,s002,1,6,0.1394,0.2343,0.0949,0.0813,0.1299,0.0486,0.0744,0.1412,0.0668,0.0863,0.9373,0.8510,0.0942,1.0896,0.9954,0.1681,0.3716,0.2035,0.1154,0.1287,0.0133,0.1272,0.1534,0.0262,0.0858,0.2528,0.1670,0.0792,0.2971,0.2179,0.0591
6,s002,1,7,0.1064,0.2069,0.1005,0.0866,0.1368,0.0502,0.0800,0.1407,0.0607,0.0789,0.7967,0.7178,0.0855,1.2005,1.1150,0.0948,0.3083,0.2135,0.1233,0.1400,0.0167,0.1318,0.1204,-0.0114,0.0782,0.1999,0.1217,0.0879,0.2907,0.2028,0.0890
7,s002,1,8,0.0929,0.1810,0.0881,0.0818,0.1378,0.0560,0.0747,0.1367,0.0620,0.0776,0.6447,0.5671,0.1373,1.1876,1.0503,0.1059,0.3139,0.2080,0.0903,0.1152,0.0249,0.1322,0.1040,-0.0282,0.0821,0.2127,0.1306,0.1006,0.2776,0.1770,0.0679
8,s002,1,9,0.0966,0.1797,0.0831,0.0771,0.1296,0.0525,0.0839,0.1425,0.0586,0.0755,0.7357,0.6602,0.0800,0.9406,0.8606,0.1027,0.2257,0.1230,0.1114,0.1260,0.0146,0.1262,0.1403,0.0141,0.0787,0.2138,0.1351,0.0882,0.2868,0.1986,0.0634
9,s002,1,10,0.1093,0.1807,0.0714,0.0731,0.1457,0.0726,0.0766,0.1241,0.0475,0.0813,0.7550,0.6737,0.0826,0.8065,0.7239,0.1156,0.3117,0.1961,0.1119,0.1785,0.0666,0.1463,0.1162,-0.0301,0.1207,0.2281,0.1074,0.1204,0.3187,0.1983,0.0866


In [5]:
%load_ext autoreload
%autoreload 2

from Biometrics import RecognitionSystem

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
inic = time.time()

param_grid = [
    {'method':['M2005'],
     'base' : ['CMU'],
    'impostor_rate' : [0.3,0.5,0.7],
    'model_size' : [40],
    'rate_external_impostor' : [0,0.5,1],
    'sampling': ['Random', 'GenFirst','ImpFirst'],
    'normalize': [False,True],
    'adaptive' : [False, 'GrowingWindow', 'SlidingWindow']},
    {'method':['M2005'],
     'base' : ['CMU'],
    'impostor_rate' : [0.3,0.5,0.7],
    'model_size' : [40],
    'rate_external_impostor' : [0,0.5,1],
    'sampling': ['SeriesAttack'],
    'len_attacks' : [5],
    'normalize': [False,True],
    'adaptive' : [False, 'GrowingWindow', 'SlidingWindow']},
]


System = RecognitionSystem(verbose=True, save_results=True)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

{'adaptive': False, 'base': 'CMU', 'impostor_rate': 0.3, 'method': 'M2005', 'model_size': 40, 'normalize': False, 'rate_external_impostor': 0, 'sampling': 'Random'}
> c:\users\feng\documents\faculdade\tcc\biometricrecognitionsystemevaluator\biometrics.py(73)_run()
-> usersDatabase, samples, decision_threshold = splitTrainTest.create(dataset=dataset, n_amostras=model_size, users_list=u_reg, classifier=method, normalize=normalize, random_state=self.random_state)
(Pdb) c
> c:\users\feng\documents\faculdade\tcc\biometricrecognitionsystemevaluator\components.py(34)create()
-> users_decision_threshold = best_threshold.run(usersData=user_information)
(Pdb) c
> c:\users\feng\documents\faculdade\tcc\biometricrecognitionsystemevaluator\components.py(77)run()
-> if self.model == 'Euclidian':
(Pdb) c
> c:\users\feng\documents\faculdade\tcc\biometricrecognitionsystemevaluator\components.py(100)run()
-> decision_threshold = self.calculate_best_threshold(userScores=genuineScores, impostorScores=impos

(Pdb) n
> c:\users\feng\documents\faculdade\tcc\biometricrecognitionsystemevaluator\components.py(64)run()
-> treino = usersData[user][:20]
(Pdb) quit


BdbQuit: 

In [8]:
np.std([0.41304347826086957, 0.532608695652174, 0.7391304347826086, 0.5543478260869565, 0.6413043478260869, 0.6847826086956522, 0.5760869565217391, 0.6956521739130435, 0.7065217391304348, 0.6304347826086957, 0.6956521739130435, 0.6195652173913043, 0.5652173913043478, 0.6086956521739131, 0.6086956521739131, 0.7391304347826086, 0.6521739130434783, 0.6630434782608695, 0.7608695652173914, 0.34782608695652173, 0.6304347826086957, 0.7608695652173914, 0.5978260869565217, 0.7065217391304348, 0.7065217391304348, 0.5978260869565217, 0.6956521739130435, 0.75, 0.7608695652173914, 0.6847826086956522, 0.7282608695652174, 0.6086956521739131, 0.6521739130434783, 0.6847826086956522, 0.4673913043478261, 0.4782608695652174, 0.8478260869565217, 0.5869565217391305, 0.6739130434782609, 0.7934782608695652])

0.10129565879885172

In [ ]:
(fim / 60) / 60

#### Manhattan

In [ ]:
df = pd.read_csv('resultados/CMU_Manhattan.csv')
df.groupby(['impostor_rate'])['FMR_mean','FNMR_mean'].mean()

#### Euclidiano

In [ ]:
df = pd.read_csv('resultados/CMU_Euclidiano.csv')
df.groupby(['impostor_rate'])['FMR_mean','FNMR_mean'].mean()

#### M2005

In [ ]:
df = pd.read_csv('resultados/CMU_M2005.csv')
df.groupby(['impostor_rate'])['FMR_mean','FNMR_mean'].mean()

__Analise__

In [ ]:
df = pd.read_csv('resultados/CMU_M2005.csv')

In [ ]:
df.head()

In [ ]:
df.groupby(['impostor_rate'])['FMR_mean','FNMR_mean'].mean()

In [ ]:
df[['impostor_rate', 'FMR_mean', 'FNMR_mean']].head(20)

In [ ]:
inic = time.time()

param_grid = [
    {'method':['M2005'],
     'base' : ['CMU'],
    'impostor_rate' : [0.3,0.5,0.7],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : [False]}
]


System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

In [ ]:
########################################

In [ ]:
inic = time.time()

param_grid_a = [
    {'method':['Euclidian'],
    'impostor_rate' : [0.3],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : [False, ]}
]

param_grid_b = [
    {'method':['Euclidian'],
    'impostor_rate' : [0.3],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : ['GrowingWindow']}
]

param_grid_c = [
    {'method':['M2005'],
    'impostor_rate' : [0.3],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : [False, ]}
]

param_grid_d = [
    {'method':['M2005'],
    'impostor_rate' : [0.3],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : ['GrowingWindow']}
]


#System = RecognitionSystem(verbose=True, save_results=False)
#System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_c)
#fim = time.time() - inic

In [ ]:
DSL.sample(10, random_state=42)

# RESULTADOS ATUAIS: melhorar --> velocidade e validacao

__Ajustando Validacao__

In [ ]:
%load_ext autoreload
%autoreload 2

from Biometrics import RecognitionSystem

In [ ]:
inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
list_test = System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_a)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

In [ ]:
inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
list_test2 =  System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_a)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

In [ ]:
inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
list_test3 = System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_a)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

In [ ]:
#checando dados do fluxo
t=list()
for i in range(len(list_test)):
    t.append(list_test3[i][4] == list_test2[i][4])
sum(t)

# fluxo sao gerados todos da mesma forma!
# modelos sao gerados todos da mesma forma!
# y_genuine esta diferente
# y_impostor esta diferente


### param_grid_a

In [ ]:
fim / 60
133.627 / 60, 194.720 / 60

In [ ]:
System.summary(System.list_params, System.list_scores)

In [ ]:
System.summary(System.list_params, System.list_scores)

In [ ]:
System.summary(System.list_params, System.list_scores)

### param_grid_c

In [ ]:
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_c)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

In [ ]:
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_c)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

In [ ]:
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_c)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

### param_grid_d

In [ ]:
inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_d)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

In [ ]:
inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_d)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

In [ ]:
inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_d)
fim = time.time() - inic
System.summary(System.list_params, System.list_scores)
fim / 60

# Adicionando Novos Classificadores

### Testando algoritmo manhattan

In [ ]:
param_grid_e = [
    {'method':['Manhattan'],
    'impostor_rate' : [0.3],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : ['GrowingWindow']}
]

inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_e)
fim = time.time() - inic

In [ ]:
System.summary(System.list_params, System.list_scores)
fim / 60

# Testando OneClassSVM

In [ ]:
param_grid_e = [
    {'method':['OCSVM'],
    'impostor_rate' : [0.3],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : ['GrowingWindow']}
]

inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_e)
fim = time.time() - inic

In [ ]:
System.summary(parameters=System.list_params, metrics=System.list_scores)

In [ ]:
System.summary(parameters=System.list_params, metrics=System.list_scores)

# Testando Mahalanobis

In [ ]:
param_grid_e = [
    {'method':['Mahalanobis'],
    'impostor_rate' : [0.3],
    'model_size' : [40],
    'rate_external_impostor' : [0],
    'sampling': ['Random'],
    'normalize': [False],
    'adaptive' : ['False']}
]

inic = time.time()
System = RecognitionSystem(verbose=True, save_results=False)
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid_e)
fim = time.time() - inic

# Param grid final

In [ ]:
from Biometrics import RecognitionSystem

In [ ]:
inic = time.time()

param_grid = [
    {'method':['Euclidian'],
    'impostor_rate' : [0.3,0.5,0.7],
    'model_size' : [40],
    'rate_external_impostor' : [0,0.5,1],
    'sampling': ['Random', 'GenFirst','ImpFirst'],
    'normalize': [False,True],
    'adaptive' : [False, 'GrowingWindow', 'SlidingWindow']},
    {'method':['Euclidian'],
    'impostor_rate' : [0.3,0.5,0.7],
    'model_size' : [40],
    'rate_external_impostor' : [0,0.5,1],
    'sampling': ['SeriesAttack'],
    'len_attacks' : [5],
    'normalize': [False,True],
    'adaptive' : [False, 'GrowingWindow', 'SlidingWindow']},
]


System = RecognitionSystem(verbose=True, save_results=True, filename='Resultados Manhattan e OCSVM')
System.fit(dataset=DSL, user_column='subject', param_grid=param_grid)
fim = time.time() - inic

In [ ]:
results = System.summary(parameters=System.list_params, metrics=System.list_scores)
results.to_csv('resultados/Euclidiano_CMU.csv', index=0)

In [ ]:
import pickle

In [ ]:
pickle.dump(System, open('resultados/euclidian_model.pkl','wb'))

In [ ]:
(fim / 60)/60

In [ ]:
fim/60